[Challenge 2](https:\8weeksqlchallenge.com\case-study-8\)

<span style="font-size:14px;"><b><i>Sales Analysis</i></b></span>

<span style="font-size:14px;"><b><i>1. What was the total quantity sold for all products?</i></b></span>

In [1]:
select 
    t1.product_name,
    t2.[total quantity]
from balanced_tree.product_details t1 join (
    select 
        prod_id,
        sum(qty) as [total quantity]
    from balanced_tree.sales
    group by prod_id
) t2 on t1.product_id = t2.prod_id;

(12 rows affected)

Total execution time: 00:00:00.074

product_name,total quantity
Navy Oversized Jeans - Womens,3843
Black Straight Jeans - Womens,3777
Cream Relaxed Jeans - Womens,3696
Khaki Suit Jacket - Womens,3749
Indigo Rain Jacket - Womens,3748
Grey Fashion Jacket - Womens,3873
White Tee Shirt - Mens,3793
Teal Button Up Shirt - Mens,3642
Blue Polo Shirt - Mens,3815
Navy Solid Socks - Mens,3779


<span style="font-size:14px;"><b><i>2. What is the total generated revenue for all products before discounts?</i></b></span>

In [2]:
with cte0 as (
	select 
		txn_id,
		sum(price*qty) as [revenue before discount per txn_id]
	from balanced_tree.sales
	group by txn_id
)

select 
	sum([revenue before discount per txn_id]) as [total revenue before discount]
from cte0;

(1 row affected)

Total execution time: 00:00:00.025

total revenue before discount
1287182


<span style="font-size:14px;"><b><i>3. What was the total discount amount for all products?</i></b></span>

In [3]:
with cte0 as
(
    select 
        txn_id,
        sum(price*qty) as [revenue before discount per txn_id]
    from balanced_tree.sales
    group by txn_id
), cte1 as 
(
    select 
        distinct(discount),
        txn_id
    from balanced_tree.sales
    where discount != 0
)

select 
    round(sum([revenue before discount per txn_id] - [revenue before discount per txn_id]*(100-cast(c1.discount as float))/100),2) as [total discount amount]
from cte0 c0 join cte1 c1 
    on c0.txn_id = c1.txn_id

(1 row affected)

Total execution time: 00:00:00.037

total discount amount
155831.54


<span style="font-size:14px;"><b><i>Transaction Analysis</i></b></span>

<span style="font-size:14px;"><b><i>1.How many unique transactions were there?</i></b></span>

In [4]:
select 
    count(distinct(txn_id)) as [number of unique transactions]
from balanced_tree.sales;

(1 row affected)

Total execution time: 00:00:00.049

number of unique transactions
2496


<span style="font-size:14px;"><b><i>2. What is the average unique products purchased in each transaction?</i></b></span>

In [5]:
with cte1 as (
	select 
		txn_id,
		cast(count(distinct(prod_id)) as  float) as [number of products per txn_id]
	from balanced_tree.sales
	group by txn_id
)
select  
	round((sum([number of products per txn_id]) / count(txn_id)),0) as [average unique products purchased in each transaction]
from cte1;

(1 row affected)

Total execution time: 00:00:00.022

average unique products purchased in each transaction
6


<span style="font-size:14px;"><b><i>3. What are the 25th, 50th and 75th percentile values for the revenue per transaction?</i></b></span>

In [7]:
with cte0 as (
	select 
		txn_id,
		sum(qty*price*(100-cast(discount as float))/100) [revenue per txn_id]
	from balanced_tree.sales
	group by txn_id
)
select 
    top 1
        round(percentile_disc(0.25) within group (order by [revenue per txn_id]) over (),2) as [25th],
        round(percentile_disc(0.5) within group (order by [revenue per txn_id]) over (),2) as [50th],
        round(percentile_disc(0.75) within group (order by [revenue per txn_id]) over (),2) as [75th]
from cte0 ;

(1 row affected)

Total execution time: 00:00:00.057

25th,50th,75th
326.18,441.48,572.8


<span style="font-size:14px;"><b style=""><i>4. What is the average discount value per transaction?</i></b></span>

In [9]:
with cte0 as
(
    select 
		distinct(txn_id) as txn_id,
		cast(discount as float) as discount
    from balanced_tree.sales
    where discount != 0
)
select 
    sum(discount) / count(txn_id) as [average discount value per transaction]
from cte0;

(1 row affected)

Total execution time: 00:00:00.022

average discount value per transaction
12.5


<span style="font-size:14px;"><b><i>5. What is the percentage split of all transactions for members vs non-members?</i></b></span>

In [10]:
with cte0 as (
	select 
		distinct(concat('Q',
				case
					when datepart(month,start_txn_time) between 1 and 3 then 1
					when datepart(month,start_txn_time) between 4 and 6 then 2
					when datepart(month,start_txn_time) between 7 and 9 then 3
					when datepart(month,start_txn_time) between 10 and 12 then 4
				end,
				'/',
				year(start_txn_time))) as start_txn_time,
		txn_id,
		member
	from balanced_tree.sales
), cte1 as (
	select 
		start_txn_time,
		cast(sum(case when member = 't' then 1 else 0 end) as float) as [number of T member transactions],
		cast(sum(case when member = 'f' then 1 else 0 end) as float) as [number of F member transactions]
	from cte0
	group by start_txn_time
), cte2 as (
	select 
		start_txn_time,
		count(txn_id) as [total transactions]
	from cte0
	group by start_txn_time
)
select 
	c1.start_txn_time,
	round(c1.[number of F member transactions] / c2.[total transactions],3) as [percentage of F member transaction],
	round(c1.[number of T member transactions] / c2.[total transactions],3) as [percentage of T member transaction]
from cte1 c1 join cte2 c2
	on c1.start_txn_time = c2.start_txn_time

(1 row affected)

Total execution time: 00:00:00.069

start_txn_time,percentage of F member transaction,percentage of T member transaction
Q1/2021,0.398,0.602


<span style="font-size:14px;"><b><i>6. What is the average revenue for member transactions and non-member transactions?</i></b></span>

In [11]:
with cte0 as
(
        select 
			distinct(concat('Q',
				case
					when datepart(month,start_txn_time) between 1 and 3 then 1
					when datepart(month,start_txn_time) between 4 and 6 then 2
					when datepart(month,start_txn_time) between 7 and 9 then 3
					when datepart(month,start_txn_time) between 10 and 12 then 4
				end,
				'/',
				year(start_txn_time))) as start_txn_time,
            txn_id,
            member,
            qty,
			price,
			cast(discount as float) as discount
        from balanced_tree.sales
),cte1 as (
	select 
		start_txn_time,
		txn_id,
		member,
		round(sum(qty*price*(100-discount)/100),2) as revenue
	from cte0
	group by start_txn_time , txn_id , member
), cte2 as (
	select 
		start_txn_time,
		sum(case when member = 't' then revenue else 0 end) as [revenue of T member],
		sum(case when member = 'f' then revenue else 0 end) as [revenue of F member],
		sum(case when member = 't' then 1 else 0 end) as [number of T member transactions],
		sum(case when member = 'f' then 1 else 0 end) as [number of F member transactions]
	from cte1
	group by start_txn_time
)
select
	start_txn_time,
	round([revenue of F member] / [number of F member transactions],3) as [average revenue of F member],
	round([revenue of F member] / [number of T member transactions],3) as [average revenue of T member]
from cte2;

(1 row affected)

Total execution time: 00:00:00.064

start_txn_time,average revenue of F member,average revenue of T member
Q1/2021,450.518,298.146


<span style="font-size:14px;"><b><i>Product Analysis</i></b></span>

**_1\. What are the top 3 products by total revenue before discount?_**

In [12]:
with cte1 as
(
    select 
        prod_id,
        sum(qty*price) as [total revenue before discount]
    from balanced_tree.sales
    group by prod_id
)

select 
    top 3 
    t1.product_name , 
    c1.[total revenue before discount]
from cte1 c1 join balanced_tree.product_details t1
    on c1.prod_id = t1.product_id
order by [total revenue before discount] desc

(3 rows affected)

Total execution time: 00:00:00.021

product_name,total revenue before discount
Blue Polo Shirt - Mens,217455
Grey Fashion Jacket - Womens,209142
White Tee Shirt - Mens,151720


<span style="font-size:14px;"><b><i>2. What is the total quantity, revenue and discount for each segment?</i></b></span>

In [13]:
select 
    d.segment_name,
    sum(s.qty) as [total quantity],
    round(sum(s.qty*s.price*(100-cast(discount as float))/100),2) as [total revenue],
    round(sum(s.qty*s.price) - sum(s.qty*s.price*(100-cast(discount as float))/100),2) as [total discount amount]
from balanced_tree.sales s join balanced_tree.product_details d
    on s.prod_id = d.product_id
group by d.segment_name;

(4 rows affected)

Total execution time: 00:00:00.033

segment_name,total quantity,total revenue,total discount amount
Jacket,11370,322373.88,44207.12
Jeans,11316,182527.36,25255.64
Shirt,11250,356092.92,49502.08
Socks,11190,270356.3,36866.7


<span style="font-size:14px;"><b><i>3. What is the top selling product for each segment?</i></b></span>

In [14]:
with cte0 as (
    select 
        d.segment_name,
        d.product_name,
        round(sum(s.qty*s.price*(100-cast(discount as float))/100),2) as revenue
    from balanced_tree.sales s join balanced_tree.product_details d 
        on s.prod_id = d.product_id
    group by d.segment_name , d.product_name
), cte1 as (
    select 
        segment_name,
        product_name,
        revenue,
        dense_rank() over (partition by segment_name order by revenue desc) as rank
    from cte0
)
select
    segment_name,
    product_name,
    revenue
from cte1
where rank = 1

(4 rows affected)

Total execution time: 00:00:00.043

segment_name,product_name,revenue
Jacket,Grey Fashion Jacket - Womens,183787.38
Jeans,Black Straight Jeans - Womens,106157.76
Shirt,Blue Polo Shirt - Mens,190674.12
Socks,Navy Solid Socks - Mens,119487.6


<span style="font-size:14px;"><b><i>4. What is the total quantity, revenue and discount for each category?</i></b></span>

In [15]:
select 
    d.category_name,
    sum(s.qty) as [total quantity],
    round(sum(s.qty*s.price*(100-cast(discount as float))/100),2) as [total revenue],
    round(sum(s.qty*s.price) - sum(s.qty*s.price*(100-cast(discount as float))/100),2) as [total discount amount]
from balanced_tree.sales s join balanced_tree.product_details d
    on s.prod_id = d.product_id
group by d.category_name;

(2 rows affected)

Total execution time: 00:00:00.025

category_name,total quantity,total revenue,total discount amount
Mens,22440,626449.22,86368.78
Womens,22686,504901.24,69462.76


<span style="font-size:14px;"><b><i>5. What is the top selling product for each category?</i></b></span>

In [16]:
with cte0 as (
    select 
        d.category_name,
        d.product_name,
        round(sum(s.qty*s.price*(100-cast(discount as float))/100),2) as revenue
    from balanced_tree.sales s join balanced_tree.product_details d 
        on s.prod_id = d.product_id
    group by d.category_name, d.product_name
), cte1 as (
    select 
        category_name,
        product_name,
        revenue,
        dense_rank() over (partition by category_name order by revenue desc) as rank
    from cte0
)
select
    category_name,
    product_name,
    revenue
from cte1
where rank = 1

(2 rows affected)

Total execution time: 00:00:00.026

category_name,product_name,revenue
Mens,Blue Polo Shirt - Mens,190674.12
Womens,Grey Fashion Jacket - Womens,183787.38


<span style="font-size:14px;"><b><i>6. What is the percentage split of revenue by product for each segment?</i></b></span>

In [17]:
with cte0 as (
    select 
        d.segment_name,
        d.product_name,
        cast(round(sum(s.qty*s.price*(100-cast(discount as float))/100),2) as float) as revenue
    from balanced_tree.sales s join balanced_tree.product_details d
        on s.prod_id = d.product_id
    group by d.segment_name , d.product_name
), cte1 as (
    select 
        segment_name,
        cast(sum(revenue) as float) as [total revenue by segment name]
    from cte0
    group by segment_name
)

select
    c1.segment_name,
    c0.product_name,
    round(c0.revenue / c1.[total revenue by segment name],2) as [percentage]
from cte0 c0 join cte1 c1
    on c0.segment_name = c1.segment_name
order by c0.segment_name , round(c0.revenue / c1.[total revenue by segment name],2) desc

(12 rows affected)

Total execution time: 00:00:00.036

segment_name,product_name,percentage
Jacket,Grey Fashion Jacket - Womens,0.57
Jacket,Khaki Suit Jacket - Womens,0.24
Jacket,Indigo Rain Jacket - Womens,0.19
Jeans,Black Straight Jeans - Womens,0.58
Jeans,Navy Oversized Jeans - Womens,0.24
Jeans,Cream Relaxed Jeans - Womens,0.18
Shirt,Blue Polo Shirt - Mens,0.54
Shirt,White Tee Shirt - Mens,0.37
Shirt,Teal Button Up Shirt - Mens,0.09
Socks,Navy Solid Socks - Mens,0.44


<span style="font-size:14px;"><b><i>7. What is the percentage split of revenue by segment for each category?</i></b></span>

In [18]:
with cte0 as (
    select 
        d.category_name,
        d.segment_name,
        cast(round(sum(s.qty*s.price*(100-cast(discount as float))/100),2) as float) as revenue
    from balanced_tree.sales s join balanced_tree.product_details d
        on s.prod_id = d.product_id
    group by d.category_name , d.segment_name
), cte1 as (
    select 
        category_name,
        cast(sum(revenue) as float) as [total revenue by category]
    from cte0
    group by category_name
)

select
    c0.category_name,
    c0.segment_name,
    round(c0.revenue / c1.[total revenue by category],2) as [percentage]
from cte0 c0 join cte1 c1 
    on c0.category_name = c1.category_name
order by c0.category_name, round(c0.revenue / c1.[total revenue by category],2) desc;

(4 rows affected)

Total execution time: 00:00:00.032

category_name,segment_name,percentage
Mens,Shirt,0.57
Mens,Socks,0.43
Womens,Jacket,0.64
Womens,Jeans,0.36


<span style="font-size:14px;"><b><i>8. What is the percentage split of total revenue by category?</i></b></span>

In [19]:
declare @total_revenue float 
select 
    @total_revenue = sum(qty*price*(100-cast(discount as float))/100)
from balanced_tree.sales;

select
    d.category_name,
    round(sum(s.qty*s.price*(100-cast(discount as float))/100) / @total_revenue,2) as revenue
from balanced_tree.sales s join balanced_tree.product_details d 
    on s.prod_id = d.product_id
group by d.category_name;

(2 rows affected)

Total execution time: 00:00:00.018

category_name,revenue
Mens,0.55
Womens,0.45


**_<span style="font-size:14px;">9. What is the total transaction “penetration” for each product? (</span>hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)_**

In [20]:
declare @total_transaction float
select 
	@total_transaction = count(distinct(txn_id))
from balanced_tree.sales
;
with cte0 as (
	select 
		s1.prod_id,
		cast(count(distinct(s1.txn_id)) as float) as [number of transactions]
	from balanced_tree.sales s1 
	where exists (
		select 
			1
		from balanced_tree.sales s2
		where s1.txn_id = s2.txn_id and s1.prod_id = s2.prod_id and s1.qty >= 1
		)
	group by s1.prod_id
)

select 
	d.product_name,
	round([number of transactions] / @total_transaction,3) as [panetration of transaction for each product]
from cte0 c0 join balanced_tree.product_details d
	on c0.prod_id = d.product_id

(12 rows affected)

Total execution time: 00:00:00.073

product_name,panetration of transaction for each product
Navy Oversized Jeans - Womens,0.509
Black Straight Jeans - Womens,0.498
Cream Relaxed Jeans - Womens,0.497
Khaki Suit Jacket - Womens,0.499
Indigo Rain Jacket - Womens,0.5
Grey Fashion Jacket - Womens,0.51
White Tee Shirt - Mens,0.507
Teal Button Up Shirt - Mens,0.497
Blue Polo Shirt - Mens,0.507
Navy Solid Socks - Mens,0.512


<span style="font-size:14px;"><b><i>10. What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?</i></b></span>

In [21]:
---  với 12 sản phẩm, tối đa có 220 sự kết hợp từ 3 phẩm khác nhau
select top 1
	s.prod_id as [product1], 
	t1.prod_id as [product2], 
	t2.prod_id as [product3], 
	count(*) as [combination count]    
from (balanced_tree.sales s join balanced_tree.sales t1 
	on t1.txn_id = s.txn_id and s.prod_id < t1.prod_id )
		join balanced_tree.sales t2 on t2.txn_id = s.txn_id and t1.prod_id < t2.prod_id
group by s.prod_id, t1.prod_id, t2.prod_id
order by count(*) desc

(1 row affected)

Total execution time: 00:00:00.143

product1,product2,product3,combination count
5d267b,9ec847,c8d436,352
